# Parte 1: DCCasas

Nota de los ayudantes: Por motivos de desempeño de las redes neuronales, recomendamos realizar esta tarea en Google Colab.

### Importando los datos

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Embedding, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from scipy.sparse import csr_matrix

In [ ]:
# Creamos una semilla para que los resultados sean replicables
n_alumno = None
np.random.seed(n_alumno)

In [ ]:
# Definimos el path al csv
# path = # Rellena con el path a la carpeta 'multimodal_house_prices/'
path = 'multimodal_house_prices/'

# Importamos el csv
meta_data = pd.read_csv(path + "data.csv")

# Veamos las primeras 5 observaciones
meta_data.head()

In [ ]:
# Preparación de datos para el MLP
X_mlp = meta_data.drop(['price', 'image_id'], axis=1)

# Definir la variable objetivo
y = np.array(meta_data["price"].tolist())

# Función para extraer características de una imagen
def image_processing(file):
    image = cv2.imread(file)

    # Obtén las dimensiones de la imagen
    height, width = image.shape[:2]

    # Verifica si la imagen ya tiene las dimensiones deseadas
    if height == 311 and width == 415:
        return image
    else:
        # Redimensiona la imagen a 415x311 con interpolación lineal
        resized_image = cv2.resize(image, (415, 311), interpolation=cv2.INTER_LINEAR)
        return resized_image
    return image

# Lista vacía para guardar los datos procesados
extracted_cnn = []

# Iterar sobre cada fila del DataFrame
for index_num, row in tqdm(meta_data.iterrows()):
    # Obtener el path de la imagen
    file_name = os.path.join(os.path.abspath(path), 'images/' +  str(row['image_id']) + '.jpg')

    # Obtener el valor objetivo (por ejemplo, el precio)
    target_value = row['price']

    # Usar la función definida arriba para procesar la imagen
    data = image_processing(file_name)

    # Guardar la imagen redimensionada
    extracted_cnn.append(data)

# Definir los datos para la CNN
X_cnn = np.array(extracted_cnn)

### Actividad 1: Importando y comprendiendo los datos

* Investiga y describe por qué el preprocesamiento es una etapa crucial para las redes neuronales. Explica cómo los pasos de preprocesamiento pueden afectar el rendimiento de un modelo.

* Si tuviéramos pocas imágenes para entrenar un modelo, ¿qué técnicas podrías usar para enriquecer el conjunto de datos? Investiga sobre el aumento de datos en imágenes.

### Actividad 2: Explorando los Multilayer Perceptron (MLP)

* Discute cómo la profundidad y el ancho (número de capas y neuronas por capa) de un MLP afectan su capacidad para aprender patrones complejos. ¿Cuáles son los desafíos relacionados con el aumento de la complejidad del modelo?

* Investiga y compara al menos dos funciones de activación no lineales diferentes utilizadas en MLPs. ¿Cómo afectan estas funciones al tipo de decisiones que puede aprender la red? Considera aspectos como la saturación y la no linealidad.

* Examina las causas y síntomas del overfitting en MLP. ¿Qué técnicas se pueden aplicar para prevenir este problema y cómo afectan el proceso de aprendizaje?

* Más allá del SGD (Stochastic Gradient Descent) y Adam, investiga sobre otro método de optimización utilizado en el entrenamiento de MLP. ¿Cuáles son sus características únicas y en qué situaciones podría ser preferible?

* Identifica y analiza un estudio de caso donde se haya utilizado un MLP para resolver un problema real. ¿Qué características del problema hicieron que un MLP fuera una buena elección y cómo se diseñó la arquitectura de la red para adaptarse a las necesidades específicas del problema?

### Implementación de un Multilayer Perceptron (MLP)

* Implementa un MLP con al menos dos capas ocultas. Entrena tu modelo en el conjunto de datos de entrenamiento y realiza ajustes en los hiperparámetros (como la tasa de aprendizaje, número de neuronas, funciones de activación) para mejorar el rendimiento del modelo.

* Evalúa el rendimiento de tu modelo en el conjunto de datos de prueba. Utiliza métricas relevantes para problemas de regresión, como el error cuadrático medio (MSE).

*  Realiza al menos dos experimentos variando la arquitectura del MLP o los hiperparámetros. Describe cómo cada cambio afecta el rendimiento del modelo y discute tus hallazgos.

* Basado en tus resultados, ¿cuáles son las características más importantes de un MLP para la predicción precisa en este conjunto de datos? ¿Cómo relacionas esto con la teoría aprendida?

* Describe los desafíos que encontraste al implementar y entrenar el MLP. ¿Cómo los superaste y qué aprendiste en el proceso? ¿en qué otros tipos de problemas crees que un MLP podría ser efectivo?

### Introducción y teoría de las CNN's

* ¿Qué es una operación convolucional? ¿Qué es un *kernel*? Utiliza estos conceptos para explicar el rol de las capas convolucionales en una CNN.

* ¿Cuál es el rol de las funciones de activación? ¿Y de las capas de *Max Pooling*?

* Quizás habrás notado que la mayoría de arquitecturas de CNN's utilizan una última capa conocida como *flatten layer*. ¿Cúal es su función? ¿Cuál es el rol de la función *softmax* en ella?

¿Cuáles son las ventajas y desventajas de las CNN's frente a las MLP's? ¿Para qué tipo de tareas suele ser útil utilizar CNN's?

### Creando y evaluando una CNN


Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

Recuerda revisar las redes, tutoriales y tips del enunciado para ayudarte a confeccionar tu propia arquitectura. A continuación, importaremos algunas librerías que podrían ser útiles para tu desarrollo.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization



from tensorflow.keras.utils import to_categorical
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from dataclasses import dataclass

Primero, preprocesaremos los datos para poder entrenar nuestra red neuronal.

Descarga el dataset si no lo has hecho:

In [ ]:
!gdown https://drive.google.com/uc?id=1078WtQHBTCe5amlDtfu3bgPbO9PFd31R
!unzip --qq multimodal_house_prices.zip

Downloading...
From: https://drive.google.com/uc?id=1078WtQHBTCe5amlDtfu3bgPbO9PFd31R
To: /content/multimodal_house_prices.zip
100% 317M/317M [00:02<00:00, 142MB/s]


Para poder usar nuestros datos en CNN, necesitamos hacer un leve preprocesamiento de los mismos. Para ello, debemos asociar las imágenes de cada casa con su respectivo precio.

In [ ]:
base_path = '/content/multimodal_house_prices'
csv_file = os.path.join(base_path, 'data.csv')
images_folder = os.path.join(base_path, 'images')

data = pd.read_csv(csv_file)

# Modificar cantidad de dataset y tamaño imágenes para ahorrar espacio en memoria.
new_data = data.sample(frac=0.6)
image_size = (156, 208)

image_data = []
prices = []

for _, row in new_data.iterrows():
    img_filename = os.path.join(images_folder, str(row['image_id']) + '.jpg')

    img = load_img(img_filename, target_size=image_size)
    img_array = img_to_array(img)

    img_array = img_array / 255.0

    image_data.append(img_array)
    prices.append(row['price'])


image_data = np.array(image_data)
prices = np.array(prices)


# Divide aquí tus datos
X_train, X_test, y_train, y_test = train_test_split(image_data, prices, test_size=0.2)

# Recuerda borrar los numpy array para ahorrar memoria.
del image_data
del prices

In [ ]:
# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees

# =====================

In [ ]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow

# =====================

In [ ]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()

# =====================

In [ ]:
# ===== COMPLETAR =====
# Grafica tu función de pérdida.

# =====================

###Comparación de modelos
A continuación, responde las siguientes preguntas relacionadas a comparar ambos tipos de redes neuronales:

* ¿Cuál de los modelos tiene un mejor *accuracy*?

* ¿Qué rangos de precios tienen un mejor rendimiento? ¿Y cuáles el peor? ¿Depende del modelo?